# find git repo
[here](https://github.com/despresj/ece_colab)

In [20]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-eb5e14ea-20a2-27a4-f285-97bfbd23707a)


In [21]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [22]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  proj_dir = "/content/drive/MyDrive/ece884_project/"
else:
  proj_dir = "../"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
df = pd.read_csv(f"{proj_dir}data_clean/clean.csv")
column_names = df.columns
df = df.to_numpy()

In [24]:
generator = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.Dense(3000, activation="selu"),
    keras.layers.Dense(3000, activation="selu"),
    keras.layers.Dense(3000, activation="selu"),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.Dense(98, activation="sigmoid")
])

In [25]:
discriminator = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.Dense(3000, activation="selu"),
    keras.layers.Dense(3000, activation="selu"),
    keras.layers.Dense(3000, activation="selu"),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(1, activation="sigmoid") # not correct output dim
])

In [26]:
gan = keras.models.Sequential([generator, discriminator])

discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [27]:
codings_size = 98
batch_size = 32 # number of data points to return
dataset = tf.data.Dataset.from_tensor_slices(df).shuffle(1000)
dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)

In [28]:
# Although this runs it still may have bugs
def train_gan(gan, dataset, batch_size, codings_size, n_epochs=50):
    generator, discriminator = gan.layers
    for epoch in range(n_epochs):
        print(f"Epoch {epoch + 1} of {n_epochs}")
        for _ in range(1000):
            idx = np.random.randint(codings_size, size=100)
            X_batch = df[inx, :]
            # phase 1 - training the discriminator
            noise = tf.random.normal(shape=(100,98))
            generated_data = generator(noise)
            X_fake_and_real = tf.concat([generated_data, X_batch], axis=0)

            y1 = tf.constant([[0.]] * noise.shape[0] + [[1.]] * X_batch.shape[0])
            y1 = np.reshape(y1, (200, 1))

            discriminator.trainable = True
            discriminator.train_on_batch(X_fake_and_real, y1)
            # phase 2 - training the generator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            y2 = tf.constant([[1.]] * batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y2)

In [29]:
import time

start = time.time()
train_gan(gan, df, batch_size, codings_size, n_epochs=1000)
end = time.time()

print(f"runtime: {(end - start) / 3600.0} hours")

Epoch 1 of 1000


NameError: ignored

In [ ]:
import os
import re
gans_saved = os.listdir(f"{proj_dir}saved_models/")
model_number = [int(re.sub("gan", "", x)) for x in gans_saved]
last_model = max(model_number)


In [ ]:
generator.save(f"{proj_dir}saved_models/gan{last_model+1}")

# Need to plot this

In [ ]:
noise = tf.random.normal(shape=[batch_size, codings_size])
generated_data = generator(noise)
generated_data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.DataFrame(df, columns=column_names)
generated_array = generated_data[0, :]
generated_df = pd.DataFrame(np.reshape(generated_array, (1, 98)), columns=column_names)

In [ ]:
generated_df.loc[:,"hospnum"]

In [ ]:
fig, ax = plt.subplots(7, 14, figsize=(50,50))
ax = ax.ravel()

for i, column in enumerate(df.columns):
    df[column].hist(ax=ax[i], color="skyblue")
    # generated_df[:,column].hist(ax=ax[i], color="red")
    

In [ ]:
fig, ax = plt.subplots(7, 14, figsize=(50,50))
ax = ax.ravel()

for i, column in enumerate(df.columns):
    generated_df[column].hist(ax=ax[i], color="skyblue")